In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer


In [2]:
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda()
model.eval()

config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(30000, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(30000, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [31]:
def summarize(
    text, n_words=None, compression=None,
    max_length=512, min_length=200, num_beams=4, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, min_length=min_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [33]:
text = """По ленд-лизу было поставлено 8 тыс. зенитных и 5 тыс. противотанковых орудий. Всего же СССР получил 38 тыс. единиц зенитной и 54 тыс. противотанковой артиллерии. То есть доля ленд-лиза в этих типах вооружения составляла соответственно 21% и 9%. Однако если брать все советские орудия и минометы в целом (поступления за войну – 526, 2 тыс.), то доля иностранных орудий в ней составит лишь 2,7%. В годы войны СССР по ленд-лизу были переданы 202 торпедных катера, 28 сторожевых кораблей, 55 тральщиков, 138 охотников за подводными лодками, 49 десантных кораблей, 3 ледокола, около 80 транспортных судов, около 30 буксиров. Всего около 580 судов. Всего же СССР за годы войны получил 2588 судов. То есть доля ленд-лизовской техники – 22,4%. Наиболее заметными стали ленд-лизовские поставки автомобилей. Всего по ленд-лизу было поставлено 480 тыс. автомобилей (из них 85% - из США). В том числе около 430 тыс. грузовых (в основном – US 6 фирм «Студебекер» и REO) и 50 тыс. джипов (Willys MB и Ford GPW). Притом, что общие поступления автомобилей на советско-германский фронт составили 744 тыс. единиц, доля ленд-лизовской техники в советском автопарке составила 64%. Кроме того, из США было поставлено 35 000 мотоциклов. А вот поставки стрелкового оружия по ленд-лизу были весьма скромными: всего порядка 150 000 тысяч единиц. Учитывая то, что общие поступления стрелкового оружия в Красную армию за войну составили 19,85 млн. единиц, доля ленд-лизовского вооружения составляет примерно 0,75%. За годы войны в СССР по ленд-лизу было поставлено 242,3 тыс. тонн автомобильного бензина (2,7% от общего производства и поступления а/м бензина в СССР). Ситуация с авиационным бензином выглядит следующим образом: из США было поставлено 570 тыс. тонн бензина, из Британии и Канада – 533,5 тыс. тонн. Кроме того, из США, Британии и Канады было поставлено 1483 тыс. тонн светлых бензиновых фракций. Из светлых бензиновых фракций в результате риформинга производится бензин, выход которого равен примерно 80%. Таким образом, из 1483 тыс. тонн фракций может быть получено 1186 тыс. тонн бензина. То есть общие поставки бензина по ленд-лизу можно оценить в 2230 тыс. тонн. В СССР за войну было произведено около 4750 тыс. тонн авиационного бензина. Вероятно, в это число включен и бензин, произведенный из поставленных союзниками фракций. То есть производство СССР бензина из собственных ресурсов можно оценить примерно в 3350 тыс. тонн. Следовательно, доля ленд-лизовского авиационного топлива от общего количества бензина, поставленного и произведенного в СССР, равна 40%. В СССР было поставлено 622,1 тыс. тонн железнодорожных рельсов, что равно 36% от общего количества рельсов, поставленных и произведенных в СССР. В ходе войны было поставлено 1900 паровозов, в то время как в СССР за 1941-1945 годы было произведено 800 паровозов, из них в 1941 – 708. Если принять число произведенных с июня по конец 1941 года паровозов за четверть от общего объема производства, то количество паровозов, произведенных за время войны, станет равным примерно 300 штукам. То есть доля ленд-лизовских паровозов в общем объеме паровозов, произведенных и поставленных в СССР, равна приблизительно 72%. Кроме того, в СССР было поставлено 11075 вагонов. Для сравнения в 1942-1945 годах в СССР было произведено 1092 ж/д вагона. За годы войны по ленд-лизу было поставлено 318 тыс. тонн взрывчатых веществ (из них США – 295,6 тыс. тонн), что составляет 36,6% от общего производства и поставок ВВ в СССР. По ленд-лизу Советский Союз получил 328 тыс. тонн алюминия. Если же верить Б.Соколову («Роль ленд-лизу в советских военных усилиях»), который оценил советское производство алюминия за время войны в 263 тыс. тонн, то доля ленд-лизовского алюминия от общего количества алюминия, произведенного и полученного СССР, составит 55%. Меди в СССР было поставлено 387 тыс. тонн – 45% от общего производства и поставок этого металла в СССР. По ленд-лизу Союзом было получено 3606 тыс. тонн автопокрышек – 30% от общего количества покрышек, произведенных и поставленных в СССР. Было поставлено 610 тыс. тонн сахара - 29,5%. Хлопка: 108 млн. тонн – 6%. Из США в СССР в годы войны было поставлено 38,1 тыс. металлорежущих станков, из Великобритании — 6,5 тыс. станков и 104 пресса. За время войны в СССР было произведено 141 тыс. м/р станков и кузнечных прессов. Таким образом, доля иностранных станков составила в отечественном хозяйстве составила 24%. В СССР также поступило 956,7 тыс. миль полевого телефонного кабеля, 2,1 тыс. миль морского кабеля и 1,1 тыс. миль подводного кабеля. Кроме того, в СССР по ленд-лизу было поставлено 35 800 радиостанций, 5899 приемников и 348 локаторов, 15,5 млн. пар армейских ботинок, 5 млн. тонн продовольствия и проч.
"""
print(summarize(text))
# Французская Эйфелева башня достигла высоты 300 метров.

В СССР по ленд-лизу были поставлены 8 тыс. зенитных и 5 тыс противотанковых орудий, в том числе 38 тыс. противотанковых орудий. Поставки стрелкового оружия на советско-германский фронт составили примерно 150 000 тысяч единиц. За годы войны Советский Союз получил 2588 судов. Доля иностранных станков составляет лишь 2,7% от общего объема произведенных с июня по конец 1941 года паровозов равна приблизительно 300 штукам. Учитывая то, что общее количество транспортных судов за время войны составила 64%. Однако доля английского металла — 22,4%. Обратите внимание на это число: 1092 железнодорожных рельсов (в основном – US 6 фирм «Студебекер» и Ford GPW.
